# Popularity Analysis of Free-to-Play App Categories

The goal of this project is to do a basic data analysis of the summary data available from the Google Play Store and the Apple iOS Mobile App Store. This project asks the question: 'If we create a free app, what category provides the best opportunity for it to be frequently installed?'. 

These datasets are available for download here:

 * [Google Play Store Apps](https://www.kaggle.com/lava18/google-play-store-apps/home)
 * [Apple iOS Store Apps](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)
 
The basic steps are as follows:

 * Open the CSV files in the Python environment, do a cursory data exploration
 * Review dataset discussion links, check for errors, remove duplicates, remove all paid apps
 * Create frequency tables for average downloads per app by category, using reviews as a proxy
 * Research categories that could field a low production cost app, find the best option

### Data Exploration

In [10]:
from csv import reader
opened_file = open('C:\Datasets\AppleStore.csv', 'r', encoding='mbcs')
read_file = reader(opened_file)
ios_data = list(read_file)
ios_header = ios_data[0]
ios_data = ios_data[1:]

opened_file = open('C:\Datasets\googleplaystore.csv', 'r', encoding='mbcs')
read_file = reader(opened_file)
ggl_data = list(read_file)
ggl_header = ggl_data[0]
ggl_data = ggl_data[1:]

In [11]:
# creates a function to view selected rows of the dataset
# or to print the dimensions of the dataset 
# function assumes the data set does not have a header

def explore_data(dataset, start, end, print_count=False):
    data_slice = dataset[start:end]
    for row in data_slice:
        print(row)
        print('\n') # creates space between rows
    if print_count:
        print('Number of rows:', len(dataset))
        print('Number of columns', len(dataset[0]))

In [12]:
print(ggl_header)
print(ggl_data[1:5])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
[['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'], ['U Launcher Lite â€“ FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'], ['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']]


In [13]:
print(ios_header)

['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [14]:
explore_data(ggl_data,0,2,print_count=True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns 13


In [15]:
explore_data(ios_data,0,2,print_count=True)

['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']


['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']


Number of rows: 7197
Number of columns 17


We've got a general idea about what these datasets contain, now we need to clean them up a bit.

### Data Cleaning

After reading through the discussion pages connected to the datasets, it looks like there is one corrupted entry in the Google Play Store data set. We can check the length of the surrounding entries to confirm it is bad data, then remove it. 

In [19]:
print(len(ggl_data[10470]))
print(len(ggl_data[10471]))
print(len(ggl_data[10472]))
print(len(ggl_data[10473]))

print(len(ggl_data))
del ggl_data[10472]
print(len(ggl_data))

13
13
12
13
10841
10840


The discussion page related to the Google Play Store data shows there are some duplicate entries in that data set. We need to check for duplicates and remove every duplicate entry except for the most recent. We can use the number of reviews to determine the most recent data point. We should also check the iOS App data as well, even though there is nothing in the discussion page.

In [20]:
unique_apps_ggl = []
duplicate_apps_ggl = []

for i in ggl_data:
    name = i[0]
    if name in unique_apps_ggl:
        duplicate_apps_ggl.append(name)
    else:
        unique_apps_ggl.append(name)
        
print('# of unique google apps: ', len(unique_apps_ggl))
print('# of duplicate google apps: ', len(duplicate_apps_ggl))

unique_apps_ios = []
duplicate_apps_ios = []

for i in ios_data:
    name = i[0]
    if name in unique_apps_ios:
        duplicate_apps_ios.append(name)
    else:
        unique_apps_ios.append(name)
        
print('# of unique apple apps: ', len(unique_apps_ios))
print('# of duplicate apple apps: ', len(duplicate_apps_ios))

# of unique google apps:  9659
# of duplicate google apps:  1181
# of unique apple apps:  7197
# of duplicate apple apps:  0


In [21]:
for i in ggl_data:
    name = i[0]
    if name == 'Facebook':
        print(i)

['Facebook', 'SOCIAL', '4.1', '78158306', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'August 3, 2018', 'Varies with device', 'Varies with device']
['Facebook', 'SOCIAL', '4.1', '78128208', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'August 3, 2018', 'Varies with device', 'Varies with device']


It appears only the Google data set contains duplicate entries. Printing a likely duplicate shows the only column that differs is the review count. Here the examples show: 78,158,306 & 78,128,208.

Because there are 1,181 duplicate entries in the Google data set, after their removal the length of the data set should be reduced from 10,840 to 9,659.

In [22]:
reviews_max = {}

for i in ggl_data:
    name = i[0]
    n_reviews = float(i[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max.update({name:n_reviews})
        
print(len(reviews_max))

9659


Above, we created a dictionary and added entries to the 'reviews_max' dictionary. If the name was already in the dictionary, the review count was updated to the higher amount. If they were not in the dictionary, we added the name and review count.

Next, we can create a new, clean data set using an empty list and a 'ggl_added' list to check if we've already added the data. The two lists should be the same length to make sure we've added everything properly, and they should match our expected length.

In [23]:
ggl_clean = []
ggl_added = []

for i in ggl_data:
    name = i[0]
    n_reviews = float(i[3])
    if n_reviews == reviews_max[name] and name not in ggl_added:
        ggl_clean.append(i)
        ggl_added.append(name)

print(len(ggl_clean))
print(len(ggl_added))

9659
9659


Because there were no errors or duplicates in the iOS data, we can create a copy of that data set as our 'ios_clean' variable for further use.

In [24]:
ios_clean = ios_data

Both data sets also have apps that are not built primarily for English users. We want to remove these entries before doing further analysis. The easiest way to do this is to remove any row of data that has text symbols not used in English. 

In ASCII encoding, the range of commonly used English characters is 0 - 127. Some real apps may have a couple non-standard characters or emoji (🙄), so we only want to remove an entry if it contains more than three non-standard characters.

In [25]:
def char_check(string):
    error_count = 0
    for char in string:
        if ord(char) > 127:
            error_count +=1
            if error_count > 3:
                return False
    return True

In [26]:
print(len(ggl_clean))
print(len(ios_clean))

9659
7197


In [27]:
ios_english = []
ggl_english = []

for row in ggl_clean:
    name = row[0]
    if char_check(name) == True:
        ggl_english.append(row)
        
for row in ios_clean:
    name = row[2]
    if char_check(name) == True:
        ios_english.append(row)
        
print(len(ios_english))
print(len(ggl_english))

6100
9500


Lastly, we want to isolate all the free apps to be our final data set.

In [33]:
ios_free = []
ggl_free = []

for row in ggl_english:
    price = row[6]
    if price == 'Free':
        ggl_free.append(row)
        
for row in ios_english:
    price = row[5]
    if price == '0':
        ios_free.append(row)
        
print(len(ios_free))
print(len(ggl_free))

3169
8759


We should now have two data sets. Each containing the free iOS and Google Play apps that are now free from errors, duplicates, non-English apps.

### App Optimization - Frequency Tables

To optimize an app, we need to figure out what works in the app stores. Below, we build frequency tables for a few data categories. Specifically, the 'Genres' and 'Category' columns from the Google Play data set.

In [34]:
def freq_table(dataset, index):
    freq_dict = {}
    percent_dict = {}
    num_entries = 0

    for row in dataset:
        num_entries += 1
        if row[index] in freq_dict:
            freq_dict[row[index]] += 1
        else:
            freq_dict[row[index]] = 1
            
    for i in freq_dict:
        percent = (float(freq_dict[i]) / num_entries)
        percent_dict[i] = percent * 100
    return percent_dict

Next we need to view the frequency table in decending order.

In [35]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_disp = []
    for key in table:
        new_tuple = (table[key], key)
        table_disp.append(new_tuple)
        
    table_sorted = sorted(table_disp, reverse = True)
    for i in table_sorted:
        print(i)

Lets look at some summary data:

In [36]:
display_table(ggl_free, 1)
# Google Play 'Category' column

(18.929101495604524, 'FAMILY')
(9.658636830688435, 'GAME')
(8.48270350496632, 'TOOLS')
(4.646649160863112, 'BUSINESS')
(3.9388057997488297, 'PRODUCTIVITY')
(3.9159721429386916, 'LIFESTYLE')
(3.721886060052517, 'FINANCE')
(3.5506336339764815, 'MEDICAL')
(3.3337138942801694, 'SPORTS')
(3.288046580659892, 'PERSONALIZATION')
(3.2537960954446854, 'COMMUNICATION')
(3.0939604977737187, 'HEALTH_AND_FITNESS')
(2.9797922137230275, 'PHOTOGRAPHY')
(2.8085397876469917, 'NEWS_AND_MAGAZINES')
(2.6487041899760246, 'SOCIAL')
(2.34044982303916, 'TRAVEL_AND_LOCAL')
(2.249115195798607, 'SHOPPING')
(2.1463637401529856, 'BOOKS_AND_REFERENCE')
(1.8609430300262586, 'DATING')
(1.8038588880009132, 'VIDEO_PLAYERS')
(1.3814362370133577, 'MAPS_AND_NAVIGATION')
(1.23301746774746, 'FOOD_AND_DRINK')
(1.1759333257221143, 'EDUCATION')
(0.9590135860258021, 'ENTERTAINMENT')
(0.9247631008105948, 'AUTO_AND_VEHICLES')
(0.9019294440004566, 'LIBRARIES_AND_DEMO')
(0.7877611599497659, 'WEATHER')
(0.7877611599497659, 'HOUSE_AND_

In [37]:
display_table(ios_free, -5)
#iOS 'prime_genre' column

(58.53581571473651, 'Games')
(7.82581255916693, 'Entertainment')
(5.0489113284947935, 'Photo & Video')
(3.72357210476491, 'Education')
(3.2817923635216157, 'Social Networking')
(2.5244556642473968, 'Shopping')
(2.398232881035027, 'Utilities')
(2.1773430104133795, 'Sports')
(2.0511202272010096, 'Music')
(1.9880088355948247, 'Health & Fitness')
(1.7040075733669928, 'Productivity')
(1.5462290943515304, 'Lifestyle')
(1.3253392237298833, 'News')
(1.1360050489113285, 'Travel')
(1.1044493531082362, 'Finance')
(0.8520037866834964, 'Weather')
(0.8204480908804039, 'Food & Drink')
(0.5364468286525718, 'Reference')
(0.5364468286525718, 'Business')
(0.3786683496371095, 'Book')
(0.18933417481855475, 'Navigation')
(0.18933417481855475, 'Medical')
(0.12622278321236985, 'Catalogs')


It seems like games and entertainment apps are the most popular in both the iOS and Google Play store. Practical applications less so. However, all we've done is reveal the most commonly published apps, not the ones with the most users. We can find out the average number of downloads per category to draw a better conclusion.

The iOS store doesn't list a download count, and the Google Play store only lists ranges, so we're going to use the number of ratings as a proxy.

In [40]:
ios_genres = freq_table(ios_free, -5)

for genre in ios_genres:
    sum_installs = 0
    num_apps = 0
    for row in ios_free:
        if row[-5] == genre:
            installs = float(row[6])
            sum_installs += installs
            num_apps += 1
    avg_installs = (sum_installs / num_apps)
    print(genre,":",round(avg_installs))

Productivity : 21799
Weather : 54215
Shopping : 27816
Reference : 79350
Finance : 32367
Music : 58205
Utilities : 19900
Travel : 31358
Social Networking : 72917
Sports : 23009
Health & Fitness : 24038
Games : 22985
Food & Drink : 33334
News : 21750
Book : 46385
Photo & Video : 28442
Entertainment : 14365
Business : 7491
Lifestyle : 16739
Education : 7004
Navigation : 86090
Medical : 612
Catalogs : 4004


In [41]:
ggl_genres = freq_table(ggl_free, 1)

for genre in ggl_genres:
    sum_installs = 0
    num_apps = 0
    for row in ggl_free:
        if row[1] == genre:
            installs = float(row[3])
            sum_installs += installs
            num_apps += 1
    avg_installs = (sum_installs / num_apps)
    print(genre,":",round(avg_installs))

ART_AND_DESIGN : 24699
AUTO_AND_VEHICLES : 14218
BEAUTY : 7476
BOOKS_AND_REFERENCE : 73425
BUSINESS : 24240
COMICS : 45617
COMMUNICATION : 993784
DATING : 22207
EDUCATION : 56293
ENTERTAINMENT : 305206
EVENTS : 2556
FINANCE : 37601
FOOD_AND_DRINK : 58483
HEALTH_AND_FITNESS : 78671
HOUSE_AND_HOME : 27812
LIBRARIES_AND_DEMO : 10564
LIFESTYLE : 34075
GAME : 691572
FAMILY : 114156
MEDICAL : 3753
SOCIAL : 982038
SHOPPING : 226042
PHOTOGRAPHY : 404081
SPORTS : 120526
TRAVEL_AND_LOCAL : 130746
TOOLS : 308587
PERSONALIZATION : 182427
PRODUCTIVITY : 160635
PARENTING : 16913
WEATHER : 175771
VIDEO_PLAYERS : 427904
NEWS_AND_MAGAZINES : 93842
MAPS_AND_NAVIGATION : 145979


Another method for the Google Play store is to use the downloads numbers as they are and check against our previous results using the ratings.

In [42]:
ggl_genres = freq_table(ggl_free, 1)

for genre in ggl_genres:
    sum_installs = 0
    num_apps = 0
    for row in ggl_free:
        if row[1] == genre:
            installs = row[5]
            installs = installs.replace('+','')
            installs = installs.replace(',','')
            sum_installs += int(installs)
            num_apps += 1
    avg_installs = (sum_installs / num_apps)
    print(genre,":",round(avg_installs))

ART_AND_DESIGN : 1986335
AUTO_AND_VEHICLES : 654075
BEAUTY : 513152
BOOKS_AND_REFERENCE : 8329169
BUSINESS : 1712290
COMICS : 859042
COMMUNICATION : 38550548
DATING : 861410
EDUCATION : 1833495
ENTERTAINMENT : 11767381
EVENTS : 253542
FINANCE : 1365500
FOOD_AND_DRINK : 1951284
HEALTH_AND_FITNESS : 4219697
HOUSE_AND_HOME : 1385541
LIBRARIES_AND_DEMO : 649314
LIFESTYLE : 1447459
GAME : 15571587
FAMILY : 3718295
MEDICAL : 121162
SOCIAL : 23628689
SHOPPING : 7103191
PHOTOGRAPHY : 17840110
SPORTS : 3750581
TRAVEL_AND_LOCAL : 14120454
TOOLS : 10902379
PERSONALIZATION : 5240359
PRODUCTIVITY : 16787331
PARENTING : 552875
WEATHER : 5212877
VIDEO_PLAYERS : 24878049
NEWS_AND_MAGAZINES : 9626407
MAPS_AND_NAVIGATION : 4115374


This generally confirms the previous analysis. The download numbers are different, but the general order and relative magnitude are the same as our data table that used reviews as a proxy.

Let's take a look at the most popular apps in a couple categories.

### Category Analysis

Again, for the Google Play Store there are two ways to go about finding the most popular apps in each category.

In [43]:
for app in ggl_free:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                or app[5] == '500,000,000'
                                or app[5] == '100,000,000'):
        print(app[0],':',app[5])

WhatsApp Messenger : 1,000,000,000+
Messenger â€“ Text and Video Chat for Free : 1,000,000,000+
Skype - free IM & video calls : 1,000,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+


In [44]:
for app in ggl_free:
    if app[1] == 'COMMUNICATION' and int(app[3]) > 10000000:
        print(app[0],':',app[3])
    

WhatsApp Messenger : 69119316
Messenger â€“ Text and Video Chat for Free : 56646578
Skype - free IM & video calls : 10484169
LINE: Free Calls & Messages : 10790289
UC Browser - Fast Download Private & Secure : 17714850
Viber Messenger : 11335481
BBM - Free Calls & Messages : 12843436


The above two sections attempt to pull the most popular apps in the Communication category. What we see is there are some Google apps that have a lot of downloads, but do not have as many total reviews as some other apps. This could be due to them being automatic downloads by the phone manufacturer or retailer.

Either way, Communication apps seems like a crowded and highly competitive market.

In [45]:
for app in ggl_free:
    if app[1] == 'FINANCE' and (app[5] == '100,000,000+'
                                or app[5] == '50,000,000+'
                                or app[5] == '10,000,000+'):        print(app[0],':',app[5])

print("===================================")

for app in ggl_free:
    if app[1] == 'FINANCE' and int(app[3]) > 100000:
        print(app[0],':',app[3])

K PLUS : 10,000,000+
Mobile Bancomer : 10,000,000+
CASHIER : 10,000,000+
Itau bank : 10,000,000+
Cash App : 10,000,000+
Bank of Brazil : 10,000,000+
PayPal : 50,000,000+
Bank of America Mobile Banking : 10,000,000+
Wells Fargo Mobile : 10,000,000+
Capital OneÂ® Mobile : 10,000,000+
Chase Mobile : 10,000,000+
HDFC Bank MobileBanking : 10,000,000+
Google Pay : 100,000,000+
Credit Karma : 10,000,000+
K PLUS : 124424
Mobile Bancomer : 278082
SCB EASY : 112656
CASHIER : 335738
Itau bank : 957973
Nubank : 130582
IKO : 167168
VTB-Online : 138371
Banorte Movil : 111632
TrueMoney Wallet : 199684
Bank of Brazil : 1336246
Money Manager Expense & Budget : 134564
Monefy - Money Manager : 111254
Mobills: Budget Planner : 161440
MetaTrader 4 : 260547
Stocks, Forex, Bitcoin, Ethereum: Portfolio & News : 157505
Yahoo Finance : 135952
Money Lover: Expense Tracker, Budget Planner : 126447
PayPal : 659760
USAA Mobile : 100997
Bank of America Mobile Banking : 341090
Wells Fargo Mobile : 250719
Capital OneÂ

Finance apps seem to be mostly banking apps of similar popularities. Without a connected banking operations, it seems unlikely we could develop a low-cost finance app (such as a retirement planner) that would be highly downloaded.

In [46]:
for app in ggl_free:
    if app[1] == 'NEWS_AND_MAGAZINES' and (app[5] == '100,000,000+'
                                or app[5] == '50,000,000+'
                                or app[5] == '10,000,000+'):        print(app[0],':',app[5])

print("===================================")

for app in ggl_free:
    if app[1] == 'NEWS_AND_MAGAZINES' and int(app[3]) > 100000:
        print(app[0],':',app[3])

Fox News â€“ Breaking News, Live Video & News Alerts : 10,000,000+
NEW - Read Newspaper, News 24h : 10,000,000+
BBC News : 10,000,000+
CNN Breaking US & World News : 10,000,000+
BaBe - Read News : 10,000,000+
detikcom - Latest & Most Complete News : 10,000,000+
Dailyhunt (Newshunt) - Latest News, Viral Videos : 50,000,000+
Read- Latest News, Information, Gossip and Politics : 10,000,000+
Reddit: Social News, Trending Memes & Funny Videos : 10,000,000+
Opera News - Trending news and videos : 10,000,000+
Topbuzz: Breaking News, Videos & Funny GIFs : 10,000,000+
Pulse Nabd - World News, Urgent : 10,000,000+
NYTimes - Latest News : 10,000,000+
Bloomberg: Market & Financial News : 10,000,000+
News Republic : 10,000,000+
Newsroom: News Worth Sharing : 10,000,000+
SmartNews: Breaking News Headlines : 10,000,000+
Updates for Samsung - Android Update Versions : 10,000,000+
Pocket : 10,000,000+
NewsDog - Latest News, Breaking News, Local News : 10,000,000+
News by The Times of India Newspaper - 

The interesting part of the 'News and Magazines' category is that it is not currently dominated by automatic downloads such as Google News. The most reviewed and most downloaded app appears to be Fox News, a non-technology company. This is closely followed by BBC and CNN.

### Conclusion

Given that no major tech firm dominates this section with pre-installed apps, it could prove to be a good opportunity to develop a low-cost news aggregation app. 

By providing an app that simply links to other sites and streams a list of headlines and AP photos, we could limit development costs and have to opportunity for substantial downloads in the future.